# 1. 장단기 기억망(long short-term memory)

<br><br>
## 1.1 개념
순환 신경망의 단점은 한 토큰의 효과가 이동구간(보통은 앞, 뒤로 두 토큰 범위)를 벗어나는 즉시 거의 완전히 사라진다는 것이다. 주어와 서술어가 멀리 떨어져있는 경우, 이런 문제를 해결하기 위해서는 신경망이 입력 문장 전체에서 어떠한 기억을 유지할 수 있어야 한다. 이런 용도로 사용할 수 있는 것이 장단기 기억 신경망이다. 장단기 기억망의 현대적인 버전들은 일반적으로 게이트 제어 순화 단위(GRU)라는 특별한 신경망 단위를 사용한다. 

장단기 순환망의 각 층에 상태라는 개념을 도입하고 각 상태는 해당층의 기억으로 작용한다. 장단기 기억망이 정보를 상태에 저장하는데 관련된 규칙들을 사람이 미리 지정하지 않는다. 그 규칙 자체가 학습의 대상이된다. 훈련 과정에서 장단기 기억망은 입력의 목푯값을 예측하는 방법을 배우는 (단순 순환 신경망이 하는 것처럼)것과 동시에, 무엇을 기억해야 할지도 배운다. 순환 신경망에서 기억과 상태라는 개념이 도입된 덕분에 장단기 기억망은 한두 토큰 떨어진 토큰들 사이의 관계뿐만 아니라 입력 견본 전체에 걸친 토큰들 사이의 의존 관계도 배울 수 있다. 어떻게 그렇게 할 수 있을까?
 
<img src='./image/lstm.jpg' width='500' height='300'>
순환 신경망과의 중요한 차이는 한 시간 단계의 활성화 출력이 다음시간 단계의 신경망에 전달될 뿐만 아니라 기억 상태 역시 다음 시간 단계로 전달된다는 점이다. 순환층의 한 뉴런은 입력에 대한 가중치들과 그 가중치들에 대한 하나의 활성화 함수로 구성되지만, 장단기 기억망의 한 세포는 그보다 복잡하다. 순환 신경망에서처럼 LSTM층의 입력은 입력 견본의 한 토큰과 이전 시간 단계의 출력을 연결한 것이다. 그러나 입력 정보는 가중치들의 벡터가 아니라 세포로 흘러 들어가며, 세포안에서 세개의 게이트를 거치게 된다. 위의 그림에서 보듯이 세 게이트는 망각 게이트, 후보 게이트, 출력 게이트이다. 각 게이트는 그 자체로 하나의 순방향 신경망이다. 즉, 각 게이트는 가중치들과 하나의 활성화 함수로 구성되며, 훈련 과정에서 그 가중치들이 갱신된다. 엄밀히 말하면 후보 게이트는 2층 순방향 신경망이며, 따라서 한 세포의 가중치 집합은 총 4개이다. 이 가중치들과 활성화 함수들에 의해, 세포를 통과해서 기억상태에 저장되는 정보가 제어된다. 

## 1.2 LSTM 모형_매개변수

In [1]:
maxlen = 400
batch_size = 32
embedding_dims = 300
epochs = 2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 400, 50)           70200     
_________________________________________________________________
dropout (Dropout)            (None, 400, 50)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________
None


앞의 순환 신경망과 은닉층의 뉴런 개수가 변하지 않았는데도 훈련할 매개변수가 훨씬 많다. 제8장의 SimpleRNN예제에서 은닉층의 연결 가중치수는 다음과 같았다. 
- 입력 벡터(토큰을 표현하는)의 성분들과 연결된 가중치 300개
- 치우침 항에 대한 가중치 1개
- 이전 시간 단계의 뉴런 출력에 대한 가중치 50개
즉, 뉴런당 가중치는 351개였고, 뉴런이 50개이므로 총 가중치 수는 351 * 50 = 17,550개이다. 

그러나 세개의 게이트로 구성된 LSTM 세포는 이런 은닉층이 네 개 있으므로, 가중치 수는 17,550*4 = 70,200이다. 

<br>

## 1.3 기억상태
기억 상태는 뉴런 개수와 같은 차원(성분 개수)의 벡터로 표현된다. 지금 예제는 50뉴런짜리 장단기 기억망이므로, 기억 상태는 성분이 50개인 부동소수점 벡터이다. 그럼 입력이 이 게이트들을 통과하는 과정에서 무슨 일이 생기는지 짚어보자. 위의 그림은 시간 단계 t에서 한 입력이 LSTM층에 처음 들어간 상황을 보여준다. 정보는 세포 안에서 여러 경로로 나아가다 결국에는 하나로 합쳐져서 세포 밖으로 출력된다. 
1. 입력 견본의 한 토큰을 표현하는 300차원 벡터가 LSTM 세포에 입력된다. 
2. 그 벡터는 이전 시간 단계에서 출력된 벡터와 연결된다. 지금 예에서는 300차원 벡터와 50차원 벡터를 연결하므로 성분이 350개인 벡터가 된다. 그리고 많은 경우 치우침 항에 해당하는 성분 하나가 추가된다. 
3. 첫 분기접에서, 연결된 입력 벡터의 한 복사본이 망각 게이트(forget gate)에 입력된다. 
4. 장단기 기억망의 이름처럼 뭔가를 기억하기 위한 요소는 후보 게이트(candidate gate)이다. 
5. 후보 게이트를 통과한 기억 벡터는 LSTM세포의 마지막 게이트인 출력 게이트에 진입한다. 
6. 보통의 순환층의 출력처럼 LSTM 세포의 출력은 그 은닉층의 출력(시간 단계 t에서의)임과 동시에 시간 단계 t+1의 입력이 된다. 

<망각게이트>
- 망각 게이트의 목표는 이름 그대로 기억을 얼마나 잊을 것인지를 배우는 것이다. 여기서 기억을 잊는다는 것은 세포의 기억 상태 성분 중 일부를 지우는(0으로 초기화)것을 말한다. 뭔가를 잊는 다는 것은 뭔가를 기억하는 것만큼이나 중요하다. 자연어 텍스트로부터 추출한 입력 견본들은 문구, 문장, 문서 같은 일정 단위들로 나뉘기 마련이며, 입력 견본이 달라지면 잊어야 할 것이 생긴다. 예를 들어 지금 문장의 주어가 복수형이라는 정보는 그 다음 문장의 술어와는 무관하므로, 지금 문장의 처리를 마쳤다면 그런 정보는 잊어야 한다. 
- 망각 게이트는 LSTM 세포에 현재 문맥과 관련이 있는 기억, 즉 유관 기억만 들어갈 자리를 마련하는 역할을 한다. 
- 망각 게이트 자체는 그냥 하나의 순방향 신경망이며, 활성화 함수는 S자형 함수이다. 따라서 각 뉴런은 0에서 1까지의 값을 출력한다.
- 각 뉴런의 출력으로 이루어진 망각 게이트의 출력 벡터를 일종의 마스크 mask로 간주할 수 있다. 출력 벡터의 한 성분이 1에 가까울수록 그 성분에 대응되는 기억 벡터의 성분이 많이 유지되고, 0에 가까울수록 많이 삭제된다. 
<img src='./image/망각게이트.jpg' width='500' height='300'>

<후보 게이트>
- 이 게이트도 망각 게이트처럼 작은 신경망을 이용해서 기억 벡터의 성분을 강화한다. 
- 이 성분을 얼마나 강화하는지는 지금가지의 입력과 이전 시간 단계의 출력에 의존한다. 
- 후보게이트는 두개의 내부 신경망을 이용해서 두 가지 일을 수행한다. 
- 후보 게이트의 첫 신경망은 활성화 함수가 S자형 함수이다 : 이 신경망의 목표는 기억 벡터의 어떤 성분들을 강화할 것인지를 배우는 것이다. 이 신경망의 출력을 망각 게이트의 출력 마스크와 비슷하되, 망각할 성분들이 아니라 강화할 성분들을 선택한다는 점이 다르다. 
- 후보 게이트의 두번째 신경망은 기억 벡터의 성분들을 어떤 값으로 갱신할 것인지 결정한다. 둘째 부분의 신경망은 -1에서 1을 출력하는 tahn함수 (쌍곡 탄젠트함수)를 활성화 함수로 사용한다. 
- 후보 게이트는 이 두 신경망의 출력 벡터들을 성분별로 곱하고, 거기에 기억 벡터를 성분 별로 더한다. 
- 결과적으로는 장단기 기억망은 새로운 세부사항을 기억하게 된다. 
- 즉, 정리하면 후보 게이트는 어떤 성분들을 갱신할 것인지와 그 성분들을 얼마나 갱신할 것인지를 동시에 학습한다. 마스크와 갱신량을 더한 것이 새 기억 벡터(기억 상태)이다. 
<img src='./image/후보게이트.jpg' width='500' height='300'>

<출력 게이트>
- 출력 게이터 이전의 게이트들은 LSTM 세포의 상태(기억 벡터)만 갱신했다. 출력 게이트는 전체 신경망에 실제로 영향을 미친다. 
- 출력 게이트는 원래의 입력(시간 단계 t의 입력 토큰과 시간 단계 t-1의 LSTM세포 출력을 연결한 것)의 복사본을 받는다. 
- 출력 게이트는 연결된 입력 벡터의 성분들에 n개의 가중치를 곱하고 S자형 활성화 함수를 적용해서 하나의 n차원 부동 소수점 벡터를 산출한다. 
- 출력 계산 방식 자체는 단순 순환 신경망의 출력층과 동일하다. 그러나 이것이 출력 게이트의 최종 출력을 아니다. 
- 최종 출력을 지금까지 구축한 기억에 의해 선별된다. 이를 위해, 이전 게이트들에서 갱신된 기억 벡터로부터 또 다른 마스크를 생성한다. 
- 이 마스크는 기억 벡터의 각 성분에 tanh 함수를 적용해서 만들어낸다. 따라서 마스크는 각 성분이 -1에서 1까지의 부동소수점 값인 n차원 벡터이다. 
- 이 벡터를 앞에서 말한 출력 벡터(S자형 함수로 얻은)와 성분별로 곱한 것이 출력 게이트의 최종 출력 벡터이자. 시간 단계t에서의 이 세포 전체 출력이다. 
<img src='./image/출력게이트.jpg' width='500' height='300'>

# 2. LSTM 파이썬 구현

### 예제 - 영화평 원본 텍스트

## 2.1 data load

In [2]:
import re
import tarfile
import tqdm
import glob
import os
from random import shuffle

from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors

import numpy as np  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,SimpleRNN, Bidirectional, Flatten, Dropout

In [3]:
def pre_process_data(filepath):
    positive_path = os.path.join(filepath,'pos')
    negative_path = os.path.join(filepath,'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    
    for filename in glob.glob(os.path.join(positive_path,'*.txt')):
        with open(filename, 'r',encoding='UTF8') as f:
            dataset.append((pos_label, f.read()))
            
    for filename in glob.glob(os.path.join(negative_path,'*.txt')):
        with open(filename, 'r',encoding='UTF8') as f:
            dataset.append((neg_label, f.read()))
            
    shuffle(dataset)
    return dataset

dataset = pre_process_data('C:/Users/today/NLP/data/aclImdb/train')

In [4]:
dataset[0]

(1,
 'Warning: contains a spoiler. Corny plot and in many cases terrible acting. Fontaine is great, but some others, particularly Richard Ney, Ivy\'s husband, are exceedingly wooden. Ney lies in bed, dying of arsenical poisoning, with every hair in place. Yet the movie is so juicy and so suspenseful. More faithful to the book than most movies of its era. Casting Joan Fontaine as a poisoner (and an adulteress, which was just as shocking then - I\'m not kidding, kids) was a masterful stroke. She\'s just her usual Joan Fontainey self. As murderers were supposed to, she dies by falling "feet foremost through the floor into an empty space."')

dataset의 각 튜플(두값쌍)은 분류명과 영화평으로 이루어지는데, 분류명은 영화평에 담긴 감정을 나타낸다. 1은 긍정적 감정, 0은 부정적 감정이다. 

## 2.2 tokenize, vectorization

In [5]:
# 미리 학습한 word_embedding을 불러오기
word_vectors = KeyedVectors.load_word2vec_format('C:/Users/today/NLP/word_embedding/GoogleNews-vectors-negative300.bin', binary=True, limit=200000)

# 단어를 토큰화하고 그 토큰들로부터 단어 벡터들을 생성하는 함수이다. 
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    for sample in dataset:
        # 각 리뷰를 토큰으로 분리하고
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        # 각 토큰에 대해서
        for token in tokens:
            try:
                # 미리 학습해둔 단어벡터에 존재하는 단어이면 단어벡터값을 추출
                sample_vecs.append(word_vectors[token])

            except KeyError:
                # 구글 word2vec 어휘에 없는 토큰도 있을테니까
                pass  # No matching token in the Google w2v vocab
        vectorized_data.append(sample_vecs)

    return vectorized_data

In [6]:
# 편의를 위해 목푯값 0들과 1들을 뽑아서 해당 훈련 견본과 같은 순서로 담아두기로 한다.

def collect_expected(dataset):
    # 자료 집합에서 목푯값들만 따로 뽑아 담는다. 추출된 목푯값들은 해당 견본들과 같은 순서
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [7]:
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

In [11]:
len(vectorized_data[0][0]) # 한 토큰 당 300차원의 단어 벡터로 표현됨.

300

## 2.3 data split

In [12]:
# 전처리된 자료 집합의 80%를 훈련용으로, 20%를 시험용으로 사용한다.
split_point = int(len(vectorized_data)*.8)

x_train = vectorized_data[:split_point]
x_test = vectorized_data[split_point:]

y_train = expected[:split_point]
y_test = expected[split_point:]

In [8]:
# 케라스는 입력 길이 정규화를 위한 pad_sequence메서드가 있다.
# 이 메서드는 스칼라열에만 작동하는데, 지금 예의 입력은 벡터열이다.
# 그래서 지금 자료에 맞는 입력 채우기 함수를 다음과 같이 정의하였다. 
# 실제로는 순환 신경망을 사용할 때는 굳이 입력 견본을 자르고 채울 필요가 없다
# 길이가 서로 다른 훈련 자료를 넣어도 순환 신경망이 주어진 입력의 토큰 수에 따라 순환층을 반복하기 때문이다.


def pad_trunc(data, maxlen):
    """ 주어진 자료 집합의 각 벡터열을 최대 길이 maxlen에 맞게 자르거나 
    영벡터들을 채운다."""
    new_data = []

    # 단어 벡터와 같은 길이의 영벡터(모든 성분이 0인 벡터)를 만든다.
    zero_vector = []
    for _ in range(len(data[0][0])):       # data[0]은 첫번째 리뷰이므로 data[0][0]은 첫번째 리뷰의 첫번째 토큰의 길이, 즉 그 토큰을 표현하는 단어벡터의 차원수가 된다.
        zero_vector.append(0.0)

    for sample in data:
 
        if len(sample) > maxlen:           # 각 리뷰의 토큰 수가 maxlen을 넘으면 
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

# 위의 함수는 [smp[:maxlen] + [[0.]*emb_dim] * (maxlen - len(smp)) for smp in data]
# 위의 한줄로 축약할 수 있다. 

In [13]:
maxlen = 400          # 견본 최대 길이(견본당 최대 토큰 수)는 400
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

# 모든 케라스가 선호하는 형태의 numpy배열로 변환한다. 
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

## 2.4 modeling & training

In [14]:
maxlen = 400
batch_size = 32       # 이 개수만큼의 견본을 처리한 후에야 오차를 역전파해서 가중치들을 갱신한다.
embedding_dims = 300  # 순환 신경망에 입력할 한 토큰 벡터의 길이(차원수)
epochs = 2            # 전체 훈련 자료 집합을 신경망에 통과시키는 주기의 횟수
num_neurons = 50

model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 400, 50)           70200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# 모형 훈련
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 289s 14ms/sample - loss: 0.4665 - accuracy: 0.7815 - val_loss: 0.3599 - val_accuracy: 0.8444
Epoch 2/2
20000/20000 [==============================] - 269s 13ms/sample - loss: 0.3506 - accuracy: 0.8522 - val_loss: 0.3426 - val_accuracy: 0.8598


검증 정확도가 같은 자료 집합에 대한 단순 순환 신경망 보다 훨씬 뛰어나다. LSTM 알고리즘의 미덕은 주어진 토큰들의 관계를 배운다는 데 있다. 

In [ ]:
# 모형 저장
model_structure = model.to_json()             # 앞의 과정을 매번 반복하지 않도록 모형의 구조와 가중치들을 저장한다.
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)

model.save_weights("lstm_weights1.h5")
print('Model saved.')

## 2.5 Predict

In [ ]:
# 모형 불러오기

from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
    json_string = json_file.read()
model = model_from_json(json_string)

model.load_weights('lstm_weights1.h5')

In [18]:
sample_1 = "I'm hate that the dismal weather that had me down for so long, when will it break! Ugh, when does happiness return? \
The sun is blinding and the puffy clouds are too thin.  I can't wait for the weekend."

vec_list = tokenize_and_vectorize([(1, sample_1)])
# 견본 문서와 함께 1이라는 값으로 튜플을 만들어서 입력한 것은 단지 이 함수가 요구때문이다. 
# 1은 그냥 의미 없는 값이며, 신경망 처리에는 포함되지 않는다. 

test_vec_list = pad_trunc(vec_list, maxlen)
# 이 예시에서도 토큰열들을 최대 길이maxlen에 맞게 절단 도는 증강한다. 

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

print("Sample's sentiment: {}".format(*model.predict_classes(test_vec)))
print("Raw output of sigmoid function: {}".format(*model.predict(test_vec)))

Sample's sentiment: [0]
Raw output of sigmoid function: [0.2900469]


predict_class() 메서드와 달리 predict()메서드는 S자형 활성화 함수의 원래 값, 즉 문턱값을 적용하기 전의 소수점 수치를 돌려준다. predict_class() 메서드는 그 수치가 0.5(문턱값)보다 크면 긍정적 분류 결과, 그렇지 않으면 부정적 분류 결과로 해석한다. 

**모형이 잘못 분류한 견본들도 유심히 볼 필요가 있다. 만일 S자형 함수의 값이 0.5에 가깝다면, 해당 문장에 대해 모형이 그냥 동전을 던져서 결정한 것이라 할 수 있다. 해당 문장을 살펴보고 직관과 상식에 의존하지 말고 통계학적으로 문장을 살펴봐야한다. 모형이 이전에 살펴본 문장들이 자주 나오지 않은 단어가 현재 문장에 있는가? 그런 단어들이 말뭉치 전체에서 드문가, 아니면 단어 내장을 위해 언어 모형을 훈련하는데 쓰인 말뭉치가 드문가? 견본의 모든 단어가 모형의 어휘에 들어 있는가?**


# 3. 모형 학습 시 고려해야 할 사항

## 3.1 maxlen 매개변수에 대해

사실 위의 예제에서 각 견본을 절단하거나 증강해서 길이가 딱 400토큰이 되게 만들어 주었다. 이는 합성곱 신경망에서는 꼭 필요한 부분이다. 합성곱 필터들이 항상 일정한 길이의 벡터를 훑게 하기 위한 것이자 합성곱 층이 항상 일정한 길이의 벡터를 출력하게 하기 위한 것이다. 순환 신경망들 역시, 분류를 위한 순방ㅎㅇ 층에 입력할 수 있도록 고정길이 생각 벡터를 산출했다. **생각벡터의 길이가 일정하다는 것은 순환층을 펼치는 시간 단계의 수(토큰 개수)가 일정하다는 뜻이다.** 위의 예제에서 순환층을 펼치는 시간 단계수를 400으로 잡은 것은 잘한 일이었을까? 이를 가늠하기 위해 몇가지 수치를 뽑아보자.

In [20]:
def test_len(data, maxlen):
    total_len = truncated = exact = padded = 0
    for sample in data:
        total_len += len(sample)
        if len(sample) > maxlen:
            truncated += 1
        elif len(sample) < maxlen:
            padded += 1
        else:
            exact +=1 
    print('Padded: {}'.format(padded))
    print('Equal: {}'.format(exact))
    print('Truncated: {}'.format(truncated))
    print('Avg length: {}'.format(total_len/len(data)))

dataset = pre_process_data('C:/Users/today/NLP/data/aclImdb/train')
vectorized_data = tokenize_and_vectorize(dataset)
test_len(vectorized_data, 400)

Padded: 22560
Equal: 12
Truncated: 2428
Avg length: 202.43204


In [21]:
len(dataset)

25000

전체 25000개 입력 견본에 대해서 400 maxlen을 맞추기 위해 길이를 더해준 견본이 22560개나 되고 사실상 입력 견본의 평균 길이가 202개 이므로 **maxlen을 200**으로 줄여서 장단기 기억망을 다시 훈련해 볼 필요가 있다. 

In [26]:
maxlen = 200
batch_size = 32         
embedding_dims = 300
epochs = 2

dataset = pre_process_data('C:/Users/today/NLP/data/aclImdb/train')
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

split_point = int(len(vectorized_data)*.8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200, 50)           70200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200, 50)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 10001     
Total params: 80,201
Trainable params: 80,201
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 154s 8ms/sample - loss: 0.4731 - accuracy: 0.7744 - val_loss: 0.3773 - val_accuracy: 0.8344
Epoch 2/2
20000/20000 [==============================] - 133s 7ms/sample - loss: 0.3614 - accuracy: 0.8463 - val_loss: 0.3493 - val_accuracy: 0.8538


**훈련 시간**

훈련이 훨씬 빨리 끝났지만, 정확도가 조금 떨어졌다. 견본 길이가 절반이 되었으므로 순방향으로 계산할 시간 단계들과 역방향으로 오차를 전파할 시간 단계들이 절반이 되었기 때문에 전체적인 훈련 시간이 절반 이상 감소했다. 

**정확도**

정확도가 개선되지 않은 주된 이유는 두 모형 모두 드롭아웃 층이 있기 때문이다. 두 모형 모두 애초에 드롭아웃 층이 과대적합을 줄여 주기 때문에 과대적합 방지에 대한 입력 차원 감소의 효과가 별로 없었고, 입력 차원 감소에 의한 자유도 감소 또는 가중치 갱신 횟수 감소가 정확도에 악영향을 미쳤다고 할 수 있다. 

**정리하자면 크기를 줄인 장단기 기억망은 더 많은 것을 배우는 것이 아니라 더 빨리 배울 뿐이다. 그러나 이 논의에서 기억해야 할 것은 시험용 입력 견본의 길이와 훈련용 입력 견본의 길이가 조화를 이루어야 한다는 점이다**

## 3.2 미지의 토큰을 제거함에 따른 문제
<br><br>
자료를 처리할 때 가장 큰 문제는 미지의 토큰을 폐기하는 것일 것이다. 여기서 미지의 토큰이란 미리 훈련된 word2vec모형에 없는 토큰을 말한다. 

다음과 같은 부정적 영화평을 생각해보자.
I don't like movie
만약 word2vec에 don't라는 단어가 없어서 신경망이 이 단어를 폐기해 버린다면 문장의 의미는 뒤집힌다. 

미지의 토큰을 폐기하는 것도 유효한 전략이긴 하지만, 다른 방법들도 존재한다.
1. 예를 들어 말뭉치의 모든 토큰에 대해 단어 벡터가 존재하는 기존 단어 내장을 구하거나 새로 훈현할 수도 있다.
하지만 이는 계산 요구량이 너무 높기 때문에 비용이 들지 않으면서 꽤 괜찮은 결과를 내는 접근 방식들이 있다. 
2. 미지의 토큰을 새로운 벡터 표현으로 대체한다. 
사실 사람 관점에서는 이런 접근 방식이 전혀 말이 되지 않는다. 하지만 모형은 미지의 토큰들을 그냥 폐기할 때와 비슷한 방식으로 이런 문제점을 무마한다. **여기서 핵심은 신경망이 훈련 집합의 모든 문장을 명시적으로 모형화하지 않는다는 것이다.** 이보다 좀 더 흔히 쓰는 또 다른 접근은 
3. 단어 벡터 어휘에 없는 모든 토큰을 하나의 특별한 토큰으로 대체하는 것이다. 
그 토큰을 흔히 'UNK'로 표기한다. 이 토큰에 해당하는 벡터는 미리 훈련된 단어 내장 자체에 설정되어 있을 수도 있고 알려진 단어 벡터에서 최대한 멀리 떨어진 벡터를 선택할 수도 있다. **토큰열 길이를 맞추기 위해 채워진 영벡터들과 마찬가지고, 훈련 과정에서 신경망을 이런 미지 토큰들에 주의를 빼앗기지 않고 좀 더 의미 있는 토큰들에 주목하는 방법을 배우게 된다.**

# 4. LSTM을 이용한 글자 기반 텍스트 생성
<br>

## 4.1 글자


## 4.2 텍스트 생성
마르코프 연쇄 기법이 1-그램, 2-그램, n-그램 다음에 특정 단어가 나타날 확률에 기초해서 단어들을 선택해 나감으로써 하나의 문장을 생성하는 것과 비슷하게 LSTM 모형은 조금 전에 본 단어들에 기초해서 그다음 단어의 확률을 학습할 수 있다. 그러나 마르코프 연쇄와는 달리 장단기 기억망은 기억 능력까지 갖추고 있기 때문에 현재 문맥을 벗어나지 않는 적절한 단어를 선택할 수 있다. 

텍스트 생성을 하도록 모형을 훈련하려면 어떻게 해야할까? 장단기 기억망을 훈련해서 얻는 진정한 성과는 예측 능력이 아니라 LSTM세포 자체이다. **언어의 일반적인 표현은 미리 부여된 분류명(목푯값)에 의존하지 않고 훈련 견본들 자체에서 배울 수 있다.** 분류 과제에서는 입력 견본에 대한 신경망의 예측 결과를 그 견본에 미리 부여해 둔 분류명과 비교해야 했지만, 지금 과게에서는 모형이 입력 견본의 각 토큰에 대해 그다음에 나올 토큰을 예측하고 그것을 입력 견본에 있는 다음 토큰과 비교하면 된다. 6장에서 사용한 단어 벡터 내장 접근 방식과 매우 비슷하다. 

텍스트 생성에서는 마지막 시간 단계에서 나온 생각벡터가 아니라 각 시간 단계가 산출한 출력이 중요하다. 훈련 과정에서는 각 시간 단계의 오차를 첫 시간 단계까지 역전파한다. 한 입력 견본 전체의 오차를 모든 시간 단계에 적용하는 것이 아니라 각 시간 단계에서 그 자신의 오차를 사용한다는 점이 중요하다. 
<img src='./image/텍스트생성.jpg' width='500' height='300'>

# 5. LSTM 텍스트 생성 구현

**일반적인 언어 모형을 위해서는 문체와 어조가 일관된 견본들로 이루어진 자료 집합을 사용하거나 충분히 큰 자료 집합을 사용해야 한다.** 다음 케라스의 예제는 프리드리히 니체의 저작에서 추출한 견본들을 사용한다. 독특한 문체를 가지고 있으면서도 좀 더 접근하기 쉬운 작가로 윌리엄 셰익스피어 작품을 사용하기로 하자. 

## 5.1 data load

In [29]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\today\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

In [30]:
from nltk.corpus import gutenberg

print(gutenberg.fileids())
# nltk 패키지는 구텐베르크의 일부 텍스트를 제공하고 셰익스피어의 희곡이 세편 있다. 
# 이번 예제에서는 이들을 연결해서 하나의 거대한 문자열을 만든다. 

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


## 5.2 preprocessing

In [33]:
# 구텐베르크 말뭉치의 모든 셰익스피어 희곡을 하나로 연결한다. 
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))                           # 반복하는 글자를 제거하기 위해 set을 사용
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))  # 원핫 부호화 과정에서 참조할 문자 대 색인 사전을 만든다. 
indices_char = dict((i, c) for i, c in enumerate(chars))  # 반대 방향의 사저느 원핫 벡터를 다시 문자로 복원할 때 참조할 사전도 만든다. 

corpus length: 375542
total chars: 50


In [38]:
print(chars)

['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ']


In [40]:
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '&': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '9': 17, ':': 18, ';': 19, '?': 20, '[': 21, ']': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'i': 31, 'j': 32, 'k': 33, 'l': 34, 'm': 35, 'n': 36, 'o': 37, 'p': 38, 'q': 39, 'r': 40, 's': 41, 't': 42, 'u': 43, 'v': 44, 'w': 45, 'x': 46, 'y': 47, 'z': 48, 'æ': 49}


In [41]:
# 원본 텍스트를 문자 수준에서 분해해서 고정 길이 문자 순차열들을 만든다. 
# 자료 집합의 크기를 키우고 일관된 패턴들을 좀 더 잘 포착하기 위해, 
# 텍스트에서 일련의 견본을 서로 겹치게 추출한다. 
maxlen = 40
step = 3
sentences = []
next_chars = []
# 반복마다 세 문자 나아가서 40문자를 추출한다. 
# 따라서 서로 겹치는, 그러나 동일하지 않은 훈련 견본들이 만들어진다. 
for i in range(0, len(text) - maxlen, step): 
    sentences.append(text[i: i + maxlen])   # 현재 위치에서 maxlen개의 문자를 추출해서 훈련 집합에 추가
    next_chars.append(text[i + maxlen])     # 그 다음 문자, 즉 현재 견본의 기대출력을 따로 저장해 둔다.
print('nb sequences:', len(sentences))

nb sequences: 125168


세편의 희곡에서 총 125168개의 훈련 견본을 얻었으며, 각 견본의 다음 문자들로 추출했다. 

In [43]:
sentences[:2]

['[the tragedie of julius caesar by willia',
 'e tragedie of julius caesar by william s']

In [44]:
next_chars[:2]

['m', 'h']

## 5.3 make dataset format

In [45]:
# 각 견본의 문자들을 원핫 벡터로 만들어서 목록 X에 추가한다. 
# 각 견본의 정답에 해당하는 그 다음 문자도 원핫 벡터로 만들어서 목록 y에 추가한다. 

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [49]:
sentences[0]

'[the tragedie of julius caesar by willia'

In [48]:
# 첫번째 입력 견본의 첫번째 글자인 [를 나타내는 원핫 벡터
X[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [50]:
# 전체 글자 개수가 50개니까
len(X[0][0])

50

## 5.4 modeling & training

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))   # 뉴런의 개수를 128로 늘렸음.
model.add(Dense(len(chars)))                             # 문자 예측을 위해 밀집층을 추가. 이 밀집층은 모든 가능한 문자에 대한 확률분포를 출력한다. 
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation (Activation)      (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________
None


- RMSprop는 '주어진 가중치에 대한 최근 기울기 크기들의 이동 평균'으로 가중치에 대한 학습 속도를 조정함으로써 학습 과정을 최적화 한다. 
- 최소화 할 손실 함수도 이전과 다르다. 이전 예제는 출력층에 있는 하나의 뉴런이 예/아니요의 결과만 예측해야 한다. 그래서 출력층이 Dense(1)에서 Dense(len(chars))로 바뀌었다. 
- 즉, 이 출력층은 하나의 50차원 벡터를 산출하며, 여기에 소프트 맥스 함수가 활성화 함수로서 적용된다. 
- 출력층인 50차원 벡터의 각 성분은 해당 글자가 기대 글자일 확률이고, 그 성분들은 모두 더하면 1이다. 
- 손실함수는 범주형 교차 엔트로피 categorical_crossentropy이다. 훈련 과정에서는 이 손실 함수를 이용해서 확률분포 벡터와 기대 문자 원핫 벡터의 차이를 최소화 한다. 
- 이번에는 드롭아웃이 없는데 이번 예제는 주어진 훈련 집합에 특화된 모형을 만드는 것이므로 과대 적합은 문제가 되지 않으며, 오히려 바람직하다. 

In [52]:
epochs = 6
batch_size = 128

model.fit(X, y,batch_size=batch_size,epochs=epochs)

Train on 125168 samples
Epoch 1/6
125168/125168 [==============================] - 251s 2ms/sample - loss: 2.1017
Epoch 2/6
125168/125168 [==============================] - 240s 2ms/sample - loss: 1.7053
Epoch 3/6
125168/125168 [==============================] - 243s 2ms/sample - loss: 1.5896
Epoch 4/6
125168/125168 [==============================] - 237s 2ms/sample - loss: 1.5246
Epoch 5/6
125168/125168 [==============================] - 243s 2ms/sample - loss: 1.4803
Epoch 6/6
125168/125168 [==============================] - 255s 2ms/sample - loss: 1.4493


## 5.5text generation
학습된 모형을 이용해서 희곡을 생성해보자. 출력 벡터는 50가지의 출력 가능한 문자들에 대한 하나의 확률분포를 서술하는 50차원 벡터이므로, 텍스트를 생성하려면 그 확률분포에서 문자를 추출(sampling)해야한다. 

In [53]:
import random

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

출력 벡터의 최대 성분은 신경망이 예측한 다음 문자, 즉 입력된 문자들 다음에 나올 가능성이 가장 크다고 판단한 문자에 해당한다. 그런데 항상 최대 확률에 해당하는 문자만 선택해서 텍스트를 생성한다면 그냥 입력 텍스트를 그대로 재현할 뿐이다. **새로운 텍스트를 만들려면 어느 정도의 무작위성이 필요하다.**  

In [55]:
import sys

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(30):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "sse) that thou might'st not loose the du"
sse) that thou might'st not loose the dues the plesse

   ham. the str

----- diversity: 0.5
----- Generating with seed: "sse) that thou might'st not loose the du"
sse) that thou might'st not loose the dues honor the word

   pol. and

----- diversity: 1.0
----- Generating with seed: "sse) that thou might'st not loose the du"
sse) that thou might'st not loose the dust my lord

   banq. go found



# 요약!

- 기억 단위를 이용해서 정보를 기억하는 능력을 갖춘 모형은 순차열 도는 시계열 자료를 좀 더 정확하고 일반적인 방식으로 처리할 수 있다. 
- 더 이상 중요하지 않은 정보를 잊는 것도 중요하다.
- 이후의 입력을 위해 새로운 정보를 모두 유지할 필요는 없다. 일부만 유지하면 되며, 장단기 기억망은 어떤 정보를 유지해야 하는지를 훈련을 통해 배운다. 
- 다음에 올 토큰이 무엇인지 예측할 수 있다면, 확률분포에 근거해서 새로운 텍스트를 생성할 수 있다. 
- 단어 기반 모형에 비해 문자 기반 모형은 더 작고 집중된 말뭉치로도 효율적이고 성공적으로 훈련 할 수 있다. 